# Segmenting and Clustering Neighborhoods in Toronto using K-Means
---------------------
~Apoorva Saxena
---------------------

This notebook contains 3 parts as mentioned below:

1. Part 1: _For this part, we will be scraping data from wikipedia to pandas dataframe and cleaning the data_
2. Part 2: _Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize         the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood._
3. Part 3: _Explore and cluster the neighborhoods in Toronto._

#### Importing Libraries

In [103]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner

from sklearn.cluster import KMeans # import k-means from clustering stage
from sklearn.preprocessing import LabelBinarizer # Binarize labels in a one-vs-all fashion

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# backend for rendering plots within the browser
%matplotlib inline 

import requests # library to handle requests

# tranforming json file into a pandas dataframe library
import json
from pandas.io.json import json_normalize

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import folium # map rendering library

from bs4 import BeautifulSoup # Library for parsing HTML and XML documents

print('Libraries imported')

Libraries imported


## __Part-1__
_For this part, we will be scraping data from wikipedia to pandas dataframe and cleaning the data_

#### Requesting URL and parsing the HTML using BeautifulSoup object

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')

#### Reading Wikipedia Table HTML text to Pandas Dataframe

In [3]:
table_text = soup.find("table",{"class":"wikitable sortable"})
df = pd.read_html(str(table_text))[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Cleaning the Table (removing cells with a Borough that is Not assigned)

In [4]:
df_wiki = df[df.Borough != 'Not assigned'].reset_index(drop=True)

In [5]:
df_wiki.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Checking whether Borough or Neighbourhood columns have Not Assigned value

In [6]:
for i in range(len(df_wiki)):
    if 'Not assigned' in (df_wiki.Borough.values[i], df_wiki.Neighbourhood.values[i]):
        print('At index',i,'Borough / Neighbourhood contains',df_wiki.Borough.values[i])
print('Not assigned cells = None')

Not assigned cells = None


####  Printing the total number of rows & columns of our dataframe.

In [7]:
print('Total number of rows =',df_wiki.shape[0])
print('Total number of columns =',df_wiki.shape[1])

Total number of rows = 103
Total number of columns = 3


## __Part-2__
_Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood._

#### Reading geographical coordinates CSV to dataframe

In [8]:
df_coord = pd.read_csv(r'C:\Users\apoor\Desktop\Coursera\Project - Capstone\Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging coordinates dataframe with our main dataframe & renaming Postal Code to Postal_Code

In [9]:
df = df_wiki.merge(df_coord, on='Postal Code')
df_neighbourhood = df.rename(columns={"Postal Code": "Postal_Code"})
df_neighbourhood.head()

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## __Part-3__
_Explore and cluster the neighborhoods in Toronto._

#### Find how many boroughs and neighborhoods

In [10]:
print('The dataframe has {} Borough and {} Neighbourhood'.format(len(df_neighbourhood['Borough'].unique()),df_neighbourhood.shape[0]))

The dataframe has 10 Borough and 103 Neighbourhood


#### We use geopy library to find the Latitude & Longitude of Toronto, Canada

In [11]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographic coordinate of Toronto, Canada are {}, {}'.format(latitude,longitude))

The geographic coordinate of Toronto, Canada are 43.6534817, -79.3839347


#### Generating map of Toronto, Canada and adding Neighborhoods and Borough as blue circle markers

In [177]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add Neighborhoods and Borough as blue circle markers
for lat, lng, borough, neighbourhood in zip(df_neighbourhood.Latitude, df_neighbourhood.Longitude, df_neighbourhood.Borough, df_neighbourhood.Neighbourhood):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.1
    ).add_to(venues_map)

venues_map # display map

#### Utilizing the Foursquare API to explore the neighborhoods and segment them

In [13]:
CLIENT_ID = 'YPBD4UIL2FU3ICVWLPAIHX25HCGWBPA2Q0K2VVUK1UPCJ5NW' # your Foursquare ID
CLIENT_SECRET = '5YGEJ440XOYGNOFARF3DGGT5YIKNLP25EBFKZWHLJX1RU0OF' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YPBD4UIL2FU3ICVWLPAIHX25HCGWBPA2Q0K2VVUK1UPCJ5NW
CLIENT_SECRET:5YGEJ440XOYGNOFARF3DGGT5YIKNLP25EBFKZWHLJX1RU0OF


#### Explore Neighborhoods in Toronto
#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *df_toronto_venues*.

In [15]:
df_toronto_venues = getNearbyVenues(names=df_neighbourhood['Neighbourhood'],
                                   latitudes=df_neighbourhood['Latitude'],
                                   longitudes=df_neighbourhood['Longitude'],
                                  )

#### What toronto_venue dataframe looks like

In [16]:
df_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


#### Dimension of toronto_venues dataframe

In [17]:
df_toronto_venues.shape

(1338, 7)

#### Let's get some basic information for toronto_venues dataframe

In [18]:
df_toronto_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Neighborhood            1338 non-null   object 
 1   Neighborhood Latitude   1338 non-null   float64
 2   Neighborhood Longitude  1338 non-null   float64
 3   Venue                   1338 non-null   object 
 4   Venue Latitude          1338 non-null   float64
 5   Venue Longitude         1338 non-null   float64
 6   Venue Category          1338 non-null   object 
dtypes: float64(4), object(3)
memory usage: 73.3+ KB


#### Value-counts is a good way of understanding how many Venue Category we have.

In [19]:
df_toronto_venues['Venue Category'].value_counts()

Coffee Shop                      96
Café                             75
Park                             45
Restaurant                       42
Pizza Place                      40
Sandwich Place                   34
Bakery                           29
Italian Restaurant               26
Japanese Restaurant              22
Bank                             21
Grocery Store                    21
Gym                              19
Sushi Restaurant                 18
Bar                              17
Fast Food Restaurant             17
Pub                              17
Breakfast Spot                   15
Pharmacy                         15
Hotel                            14
Thai Restaurant                  13
Ice Cream Shop                   13
American Restaurant              12
Yoga Studio                      12
Clothing Store                   12
Dessert Shop                     12
Diner                            12
Chinese Restaurant               11
Convenience Store           

#### Let's us count the unique values for Venue Category 

In [64]:
print('Count of Venue Category {}'.format(len(df_toronto_venues['Venue Category'].unique())))

Count of Venue Category 235


#### Let's analyze each Neighborhood

In [42]:
df_venue_dummy = pd.get_dummies(df_toronto_venues[['Venue Category']],prefix='',prefix_sep ='') # One hot encoding
df_venue_dummy = df_venue_dummy.drop(columns=['Neighborhood']) # dropping existing Neighborhood column
df_venue_dummy.insert(0,'Neighborhood',df_toronto_venues['Neighborhood']) # Inserting Neighborhood coloumn at the beginning from df_toronto_venues

In [50]:
df_venue_dummy.head() # Visualizing Dataframe

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0

#### Shape of df_venue_dummy

In [52]:
print('Rows {}, Columns {}'.format(df_venue_dummy.shape[0],df_venue_dummy.shape[1]))

Rows 1338, Columns 235


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [128]:
df_group_neigh = df_venue_dummy.groupby(['Neighborhood']).mean().reset_index()
df_group_neigh.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.

#### Let's look at the shape of the new dataset

In [53]:
print('Rows {}, Columns {}'.format(df_group_neigh.shape[0],df_group_neigh.shape[1]))

Rows 95, Columns 235


#### Let's make a function named top_venue() which will prints venues for each neighborhood

In [83]:
def top_venue(n): # n = number of top venues ex: for top five veneus put num_top_venues = 5
    for i in df_group_neigh['Neighborhood']:
        print('---',i,'---')
        temp = df_group_neigh[df_group_neigh['Neighborhood']==i].T.reset_index()
        temp.columns = ['Venue', 'Frequency']
        temp = temp.iloc[1:]
        temp['Frequency'] = temp['Frequency'].astype('float64')
        temp = temp.round({'Frequency':1})
        print(temp.sort_values('Frequency',ascending=False).reset_index(drop=True).head(n))
        print('\n')    

#### Let's print top 5 veneues

In [178]:
top_venue(5)

--- Agincourt ---
                       Venue  Frequency
0                     Lounge        0.2
1  Latin American Restaurant        0.2
2             Breakfast Spot        0.2
3               Skating Rink        0.2
4               Liquor Store        0.0


--- Alderwood, Long Branch ---
         Venue  Frequency
0  Pizza Place        0.2
1     Pharmacy        0.1
2         Pool        0.1
3  Coffee Shop        0.1
4          Pub        0.1


--- Bathurst Manor, Wilson Heights, Downsview North ---
                        Venue  Frequency
0                 Coffee Shop        0.1
1                        Bank        0.1
2           Accessories Store        0.0
3  Modern European Restaurant        0.0
4                      Market        0.0


--- Bayview Village ---
                 Venue  Frequency
0                 Café        0.2
1  Japanese Restaurant        0.2
2   Chinese Restaurant        0.2
3                 Bank        0.2
4    Accessories Store        0.0


--- Bedford Park,

#### Putting above data into dataframe
##### Let's write a function which will sort the venues in descending order

In [92]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [137]:
num_top_venues = 10

# create columns according to number of top venues
columns = ['Neighborhood','1st Most Common Venue',
           '2nd Most Common Venue','3rd Most Common Venue', 
           '4th Most Common Venue', '5th Most Common Venue', 
           '6th Most Common Venue', '7th Most Common Venue', 
           '8th Most Common Venue', '9th Most Common Venue', 
           '10th Most Common Venue']

# create a new dataframe
df_neigh_ven_sort = pd.DataFrame(columns=columns)
df_neigh_ven_sort['Neighborhood'] = df_group_neigh['Neighborhood']

for i in range(len(df_group_neigh)):
    df_neigh_ven_sort.iloc[i, 1:] = return_most_common_venues(df_group_neigh.iloc[i, :], num_top_venues)

df_neigh_ven_sort.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,College Rec Center,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pharmacy,Gym,Sandwich Place,Pub,Pool,Dim Sum Restaurant,Deli / Bodega,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Shopping Mall,Pharmacy,Deli / Bodega,Supermarket,Sushi Restaurant,Restaurant,Convenience Store,Middle Eastern Restaurant
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Restaurant,Sandwich Place,Sushi Restaurant,Pharmacy,Pizza Place,Pub,Café,Butcher


#### Clustering neighborhoods into 6 clusters using k-means

In [135]:
num_clusters = 6

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
df_k_means = df_group_neigh.drop('Neighborhood',1)
k_means.fit(df_k_means)
labels = k_means.labels_

print(labels)

[1 4 1 1 1 1 1 1 1 1 3 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 3 1 1 1 3 1 3 1 1 1 1
 4 4 5 1 1 1 1 3 1 3 1 1 4 3 1 3 1 3 1 1 1 5 1 4 3 1 1 1 3 1 0 1 4 3 4 1 1
 4 1 1 1 1 1 1 1 1 1 1 1 4 2 1 2 1 4 1 1 3]


Correcting typing mistake (British English to US English)

In [129]:
df_copy = df_neighbourhood
df_copy = df.rename(columns={"Neighbourhood": "Neighborhood"})

In [138]:
df_neigh_ven_sort.insert(0,'Cluster Labels',labels) # Inserting Cluster Label coloumn

Merging the dataframe

In [140]:
df_merge = df_copy.merge(df_neigh_ven_sort, on='Neighborhood')
df_merge.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Park,Convenience Store,Food & Drink Shop,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Hockey Arena,Coffee Shop,French Restaurant,Financial or Legal Service,Portuguese Restaurant,Pizza Place,Donut Shop,Dog Run,Distribution Center,Dance Studio
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Theater,Café,Bakery,Pub,Breakfast Spot,Yoga Studio,Distribution Center,Spa
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Boutique,Event Space,Vietnamese Restaurant,Gift Shop,Concert Hall,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Yoga Studio,Bank,Beer Bar,Japanese Restaurant,Sandwich Place,Distribution Center,Burrito Place,Café


#### Let's visualize the clusters in the map

In [179]:
# create map
cluster_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neigh, cluster in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Neighborhood'], df_merge['Cluster Labels']):
    label = folium.Popup(str(neigh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster_map)
       
cluster_map

## Examine Clusters
Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

##### Cluster 1

In [176]:
cluster_1 = df_merge[df_merge['Cluster Labels']==0]
cluster_1.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Bar,Yoga Studio,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


##### Cluster 2

In [175]:
cluster_2 = df_merge[df_merge['Cluster Labels']==1]
cluster_2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Hockey Arena,Coffee Shop,French Restaurant,Financial or Legal Service,Portuguese Restaurant,Pizza Place,Donut Shop,Dog Run,Distribution Center,Dance Studio
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Theater,Café,Bakery,Pub,Breakfast Spot,Yoga Studio,Distribution Center,Spa
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Boutique,Event Space,Vietnamese Restaurant,Gift Shop,Concert Hall,Electronics Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Yoga Studio,Bank,Beer Bar,Japanese Restaurant,Sandwich Place,Distribution Center,Burrito Place,Café
6,M3B,North York,Don Mills,43.745906,-79.352188,1,Gym,Café,Restaurant,Coffee Shop,Beer Store,Japanese Restaurant,Bike Shop,Bubble Tea Shop,Sandwich Place,Italian Restaurant


##### Cluster 3

In [174]:
cluster_3 = df_merge[df_merge['Cluster Labels']==2]
cluster_3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,2,Park,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
61,M9N,York,Weston,43.706876,-79.518188,2,Park,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


##### Cluster 4

In [173]:
cluster_4 = df_merge[df_merge['Cluster Labels']==2]
cluster_4.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,2,Park,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
61,M9N,York,Weston,43.706876,-79.518188,2,Park,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


##### Cluster 5

In [172]:
cluster_5 = df_merge[df_merge['Cluster Labels']==4]
cluster_5.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,4,Pizza Place,Pharmacy,Bank,Gastropub,Gym / Fitness Center,Athletics & Sports,Café,Intersection,Discount Store,Dim Sum Restaurant
25,M2H,North York,Hillcrest Village,43.803762,-79.363452,4,Golf Course,Fast Food Restaurant,Dog Run,Pool,Mediterranean Restaurant,Deli / Bodega,Eastern European Restaurant,Drugstore,Donut Shop,Distribution Center
49,M9L,North York,Humber Summit,43.756303,-79.565963,4,Pizza Place,Gym,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
60,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262,4,Pizza Place,Grocery Store,Brewery,Convenience Store,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop


##### Cluster 6

In [171]:
cluster_6 = df_merge[df_merge['Cluster Labels']==5]
cluster_6.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,M9M,North York,"Humberlea, Emery",43.724766,-79.532242,5,Baseball Field,Food Service,Yoga Studio,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
97,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,5,Baseball Field,Yoga Studio,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


In [180]:
print('Number of neighborhoods in cluster-1: {}, cluster-2: {}, cluster-3: {}, cluster-4: {}, cluster-5: {}, cluster-6: {}'.format(len(cluster_1),len(cluster_2),len(cluster_3),len(cluster_4),len(cluster_5),len(cluster_6)))

Number of neighborhoods in cluster-1: 1, cluster-2: 70, cluster-3: 2, cluster-4: 2, cluster-5: 11, cluster-6: 2


#### Clusters 2 and 5 are the biggest with seventy and eleven neighborhood respectively. 
#### Cluster 1 has only one neighbourhood and clusters 3, 4 and 6 have just two neighbourhood each.